In [1]:
#%% Importação dos pacotes

import pandas as pd
# import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, \
    accuracy_score
from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical
import lightgbm as lgb

In [2]:
#%% Carregar os dados
X_train = pd.read_pickle('X_train.pkl')
y_train = pd.read_pickle('y_train.pkl')['label']
X_test  = pd.read_pickle('X_test.pkl')
y_test  = pd.read_pickle('y_test.pkl')['label']


In [3]:
#%% Verificar as categorias das labels
níveis = y_test.cat.categories
print(níveis)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)


Index(['LAYING', 'SITTING', 'STANDING', 'WALKING', 'WALKING_DOWNSTAIRS',
       'WALKING_UPSTAIRS'],
      dtype='object')
(7352, 562) (7352,) (2947, 562) (2947,)


In [4]:
#%% Definir o espaço de busca de hiperparâmetros para o LightGBM
param_space = {
    'n_estimators': Integer(50, 500),  # Número de árvores
    'max_depth': Integer(3, 15),       # Profundidade máxima das árvores
    'learning_rate': Real(0.01, 0.3, 'log-uniform'),  # Taxa de aprendizado
    'num_leaves': Integer(20, 100),    # Número máximo de folhas
    'min_child_samples': Integer(10, 100),  # Número mínimo de amostras por folha
    'subsample': Real(0.5, 1.0),       # Subamostragem de dados
    'colsample_bytree': Real(0.5, 1.0),  # Subamostragem de features
    'reg_alpha': Real(0, 1),           # Regularização L1
    'reg_lambda': Real(0, 1),          # Regularização L2
    'boosting_type': Categorical(['gbdt', 'dart'])  # Tipo de boosting
}


In [5]:
#%% Configurar o modelo LightGBM
lgb_model = lgb.LGBMClassifier(random_state=2244000, verbose=-1)


In [6]:
#%% Configurar o Bayesian Search
bayes_search = BayesSearchCV(
    estimator=lgb_model,
    search_spaces=param_space,
    n_iter=5,  # Número de iterações
    cv=2,       # Número de folds na validação cruzada
    scoring='accuracy',
    n_jobs=-1,  # Usar todos os núcleos do processador
    verbose=1,
    random_state=2244000
)

In [7]:
#%% Executar o Bayesian Search
tempo_ini = pd.Timestamp.now()  # Início do cronômetro
bayes_search.fit(X_train, y_train)
tempo_fim = pd.Timestamp.now()  # Fim do cronômetro
print(f"Tempo de execução: {tempo_fim - tempo_ini}")


Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Tempo de execução: 0 days 00:00:41.072518


In [8]:
#%% Melhores hiperparâmetros encontrados
print("Melhores hiperparâmetros:", bayes_search.best_params_)


Melhores hiperparâmetros: OrderedDict([('boosting_type', 'gbdt'), ('colsample_bytree', 0.882556913536217), ('learning_rate', 0.013342604749226673), ('max_depth', 7), ('min_child_samples', 85), ('n_estimators', 355), ('num_leaves', 47), ('reg_alpha', 0.5039269732516942), ('reg_lambda', 0.9184079533796609), ('subsample', 0.9531043958851619)])


In [9]:
#%% Gerar as previsões do modelo
pred = bayes_search.predict(X_test)


In [10]:
#%% Gerar a matriz de confusão e estatísticas
cm = confusion_matrix(y_test, pred)
print("Matriz de Confusão:")
print(cm)
print("\nRelatório de Classificação:")
print(classification_report(y_test, pred))


Matriz de Confusão:
[[537   0   0   0   0   0]
 [  0 429  61   0   0   1]
 [  0  29 503   0   0   0]
 [  0   0   0 490   2   4]
 [  0   0   0   8 381  31]
 [  0   0   0  31   6 434]]

Relatório de Classificação:
                    precision    recall  f1-score   support

            LAYING       1.00      1.00      1.00       537
           SITTING       0.94      0.87      0.90       491
          STANDING       0.89      0.95      0.92       532
           WALKING       0.93      0.99      0.96       496
WALKING_DOWNSTAIRS       0.98      0.91      0.94       420
  WALKING_UPSTAIRS       0.92      0.92      0.92       471

          accuracy                           0.94      2947
         macro avg       0.94      0.94      0.94      2947
      weighted avg       0.94      0.94      0.94      2947



In [11]:
#%% Gerar um DataFrame temporário para avaliar o modelo
lgb_aval = pd.DataFrame({
    'pred': pred,
    'obs': y_test
})


In [12]:
#%% Função personalizada para summary (multiClassSummary equivalente)
def multiClassSummary(df, levels):
    report = classification_report(df['obs'], df['pred'], output_dict=True)
    summary = {
        'Accuracy': report['accuracy']
    }
    for level in levels:
        summary[f'{level} Precision'] = report[level]['precision']
        summary[f'{level} Recall'] = report[level]['recall']
        summary[f'{level} F1-score'] = report[level]['f1-score']
    return summary

In [13]:
#%% Calcular métricas de avaliação
metrics = pd.Series(multiClassSummary(lgb_aval, níveis))
print("\nMétricas de Avaliação:")
print(metrics)
acc_teste = accuracy_score(y_test, lgb_aval.pred)
print(f'A acurácia na base de teste foi de: {acc_teste:.2%}')


Métricas de Avaliação:
Accuracy                        0.941296
LAYING Precision                1.000000
LAYING Recall                   1.000000
LAYING F1-score                 1.000000
SITTING Precision               0.936681
SITTING Recall                  0.873727
SITTING F1-score                0.904110
STANDING Precision              0.891844
STANDING Recall                 0.945489
STANDING F1-score               0.917883
WALKING Precision               0.926276
WALKING Recall                  0.987903
WALKING F1-score                0.956098
WALKING_DOWNSTAIRS Precision    0.979434
WALKING_DOWNSTAIRS Recall       0.907143
WALKING_DOWNSTAIRS F1-score     0.941904
WALKING_UPSTAIRS Precision      0.923404
WALKING_UPSTAIRS Recall         0.921444
WALKING_UPSTAIRS F1-score       0.922423
dtype: float64
A acurácia na base de teste foi de: 94.13%
